# Question 4
## Estimating the Standard Deviation of a Prediction

Suppose we use a statistical learning method to predict the response $Y$ at a particular value of the predictor $X = x_0$. We want to estimate the **standard deviation of this prediction**, which quantifies uncertainty.

There are two main sources of uncertainty:

---

### 1. Prediction Variance (Model Uncertainty)

When we estimate the prediction function $\hat{f}(x_0)$, there's variability due to the fact that we're using a **finite training set**. This gives us **variance of the estimate** $\hat{f}(x_0)$.

#### How to estimate:

Use **bootstrapping**:

1. Repeatedly draw bootstrap samples from the training data.
2. For each bootstrap sample $b$, fit the model and compute the prediction at $x_0$:  
   $\hat{f}^{(b)}(x_0)$
3. Estimate the **standard deviation** of these predictions:

$$
\text{SD}(\hat{f}(x_0)) \approx \text{Standard deviation of } \left\{ \hat{f}^{(1)}(x_0), \hat{f}^{(2)}(x_0), \ldots, \hat{f}^{(B)}(x_0) \right\}
$$

---

### 2. Irreducible Error (Intrinsic Noise)

This is the variance of the random error $\varepsilon$, which represents the **unpredictable noise** in the data, even with the best model.

#### How to estimate:

Use the residual variance from the training data:

$$
\hat{\sigma}^2 = \frac{1}{n - d - 1} \sum_{i=1}^n (y_i - \hat{f}(x_i))^2
$$

Where:
- $n$: number of observations  
- $d$: number of predictors (degrees of freedom)  
- $\hat{f}(x_i)$: predicted value for observation $i$

---

### Total Prediction Uncertainty

If we are predicting a **new observation** $Y_0$ at $X = x_0$, then both sources of uncertainty must be considered:

$$
\text{Var}(Y_0) = \text{Var}(\hat{f}(x_0)) + \sigma^2
$$

So, the **standard deviation of the prediction** is:

$$
\text{SD}(Y_0) = \sqrt{ \text{Var}(\hat{f}(x_0)) + \hat{\sigma}^2 }
$$

---

### Summary

To estimate the prediction uncertainty at $X = x_0$:

1. **Model variance** is estimated using bootstrapped predictions.
2. **Irreducible error** is estimated using training residuals.
3. Combine both for the total standard deviation of prediction if we aim to predict a **new** $Y_0$.


# Question 9

In [7]:
from ISLP import load_data
import numpy as np
import pandas as pd


# Load Boston housing dataset from ISLP
Boston = load_data('Boston')

# (a) Estimate for the population mean of medv
mu_hat = Boston['medv'].mean()

# (b) Standard error using the theoretical formula
std_dev = Boston['medv'].std()
n = len(Boston)
se_mu_hat = std_dev / np.sqrt(n)

# (c) Bootstrap estimate of SE of the mean
def bootstrap_statistic(data, stat_fn, n_bootstrap=1000):
    estimates = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        estimates.append(stat_fn(sample))
    return np.array(estimates)



bootstrap_means = bootstrap_statistic(Boston['medv'], np.mean)
se_mu_hat_bootstrap = np.std(bootstrap_means)

# (d) 95% Confidence Interval for mean
ci_lower = mu_hat - 2 * se_mu_hat_bootstrap
ci_upper = mu_hat + 2 * se_mu_hat_bootstrap

# (e) Estimate of median
mu_med = Boston['medv'].median()

# (f) Bootstrap SE of median
bootstrap_medians = bootstrap_statistic(Boston['medv'], np.median)
se_mu_med = np.std(bootstrap_medians)

# (g) Estimate of 10th percentile
mu_10 = np.percentile(Boston['medv'], 10)

# (h) Bootstrap SE of 10th percentile
bootstrap_p10 = bootstrap_statistic(Boston['medv'], lambda x: np.percentile(x, 10))
se_mu_10 = np.std(bootstrap_p10)

# Combine all results into a DataFrame for display
summary = pd.DataFrame({
    'Statistic': ['Mean of medv', 'SE (formula)', 'SE (bootstrap)', 
                  '95% CI lower (bootstrap)', '95% CI upper (bootstrap)',
                  'Median of medv', 'SE of median (bootstrap)',
                  '10th percentile of medv', 'SE of 10th percentile (bootstrap)'],
    'Value': [mu_hat, se_mu_hat, se_mu_hat_bootstrap, 
              ci_lower, ci_upper,
              mu_med, se_mu_med, mu_10, se_mu_10]
})

print(summary)

                           Statistic      Value
0                       Mean of medv  22.532806
1                       SE (formula)   0.408861
2                     SE (bootstrap)   0.419398
3           95% CI lower (bootstrap)  21.694010
4           95% CI upper (bootstrap)  23.371603
5                     Median of medv  21.200000
6           SE of median (bootstrap)   0.383025
7            10th percentile of medv  12.750000
8  SE of 10th percentile (bootstrap)   0.508753
